# Get predictions for new data

In [1]:
import pycaret 
print(pycaret.__version__)

2.3.10


In [2]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from pycaret.regression import *

In [3]:
def download_blob(container_name, source_blob_name, destination_file_name):
  # Download the blob to a local file
    print("\nDownloading blob to \n\t" + destination_file_name)

  # Create a blob client using the local file name as the name for the blob
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=source_blob_name)

    if destination_file_name is not None: 
        with open(destination_file_name, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())

        print(
            "Blob {} downloaded to {}.".format(
                source_blob_name, destination_file_name
            )
        )

#### We establish the connection

In [4]:
## Enter connection string when running in google colab
connect_str = 'DefaultEndpointsProtocol=https;AccountName=machinelearninginsurance;AccountKey=VvrtwyVNMZg7rnks4Bfg34gQDBKgRYrbUWI/QzLt+61x+hfW7Q+GPah8NOa1dhVpA26sIhGlwe/8+AStfRIdGw==;EndpointSuffix=core.windows.net' 
print(connect_str)

# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

DefaultEndpointsProtocol=https;AccountName=machinelearninginsurance;AccountKey=VvrtwyVNMZg7rnks4Bfg34gQDBKgRYrbUWI/QzLt+61x+hfW7Q+GPah8NOa1dhVpA26sIhGlwe/8+AStfRIdGw==;EndpointSuffix=core.windows.net


In [7]:
# Create a unique name for the container
container_name = "machinelearninginsurance" 
model_name = 'lightgbm-reg-101'

##### Model Download

In [8]:
outfile_name = 'lightgbm-reg101-azure-downloaded'
model_azure_src = str(model_name)+'.pkl'
download_blob(container_name, model_azure_src, outfile_name + '.pkl')


	lightgbm-reg101-azure-downloaded.pkl
Blob lightgbm-reg-101.pkl downloaded to lightgbm-reg101-azure-downloaded.pkl.


### Forecast with new data

In [9]:
# Loading the model for predictions
azure_final_lightgbm = load_model(outfile_name)

Transformation Pipeline and Model Successfully Loaded


In [10]:
from pycaret.datasets import get_data
dataset = get_data('diamond')

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


In [11]:
data = dataset.sample(frac=0.95, random_state=786).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

In [12]:
# Predictions from deployed model
new_prediction_azure = predict_model(azure_final_lightgbm, data=data_unseen)
new_prediction_azure.head()

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price,Label
0,1.23,Very Good,G,VS1,EX,EX,GIA,8445,9119.291988
1,0.90,Fair,I,VS1,VG,EX,GIA,3526,3584.631372
2,0.77,Very Good,G,VVS1,EX,ID,AGSL,3966,3343.725163
3,1.51,Very Good,D,VS2,EX,EX,GIA,14416,14922.230858
4,2.33,Ideal,H,SI1,ID,ID,AGSL,21618,20527.248387
